In [92]:
import numpy as np
import pandas as pd

# Day ahead prices data

In [93]:
price_2015 = pd.read_csv('data/Day-ahead Prices_201501010000-201601010000.csv') 
price_2015 = price_2015.iloc[96:] # data of january 1 to 4 is missing so remove them
price_2016 = pd.read_csv('data/Day-ahead Prices_201601010000-201701010000.csv') 
price_2017 = pd.read_csv('data/Day-ahead Prices_201701010000-201801010000.csv') 
price_2018 = pd.read_csv('data/Day-ahead Prices_201801010000-201901010000.csv') 
price_2019 = pd.read_csv('data/Day-ahead Prices_201901010000-202001010000.csv') 
price_2020 = pd.read_csv('data/Day-ahead Prices_202001010000-202101010000.csv') 

In [94]:
# create 1 day ahead prices by combining all the years
day_ahead = pd.concat([price_2015, price_2016, price_2017, price_2018, price_2019, price_2020]).reset_index(drop=True)

In [95]:
day_ahead

,MTU (CET),Day-ahead Price [EUR/MWh]
0,05.01.2015 00:00 - 05.01.2015 01:00,36.56
1,05.01.2015 01:00 - 05.01.2015 02:00,34.94
2,05.01.2015 02:00 - 05.01.2015 03:00,32.19
3,05.01.2015 03:00 - 05.01.2015 04:00,28.05
4,05.01.2015 04:00 - 05.01.2015 05:00,28.04
...,...,...
52513,31.12.2020 19:00 - 31.12.2020 20:00,61.51
52514,31.12.2020 20:00 - 31.12.2020 21:00,56.79
52515,31.12.2020 21:00 - 31.12.2020 22:00,52.44
52516,31.12.2020 22:00 - 31.12.2020 23:00,51.86


In [96]:
day_ahead[day_ahead.isnull().any(axis=1)]

,MTU (CET),Day-ahead Price [EUR/MWh]
1994,29.03.2015 02:00 - 29.03.2015 03:00,NaN
10731,27.03.2016 02:00 - 27.03.2016 03:00,NaN
19468,26.03.2017 02:00 - 26.03.2017 03:00,NaN
28205,25.03.2018 02:00 - 25.03.2018 03:00,NaN
37110,31.03.2019 02:00 - 31.03.2019 03:00,NaN
45847,29.03.2020 02:00 - 29.03.2020 03:00,NaN


In [97]:
# interpolate missing day ahead price, for now using polynomial interpolation with degree 2
day_ahead['Day-ahead Price [EUR/MWh]'] = day_ahead['Day-ahead Price [EUR/MWh]'].interpolate(method='polynomial', order=2)

In [98]:
day_ahead[day_ahead.isnull().any(axis=1)]

,MTU (CET),Day-ahead Price [EUR/MWh]


# Weather data

In [99]:
weather = pd.read_json('data/parsed_weather.json')

In [100]:
weather

,date,time,tempC,windspeedKmph,winddirDegree,winddir16Point,weatherDesc,precipMM,humidity,pressure,HeatIndexC,FeelsLikeC
0,2015-01-01,0,0,12,226,SW,[Mist],0.0,96,1037,1,-3
1,2015-01-01,100,0,13,217,SW,[Mist],0.0,95,1036,0,-4
2,2015-01-01,200,0,15,209,SSW,[Clear],0.0,95,1035,0,-4
3,2015-01-01,300,0,16,200,SSW,[Clear],0.0,95,1034,0,-5
4,2015-01-01,400,0,16,201,SSW,[Clear],0.0,94,1035,0,-5
...,...,...,...,...,...,...,...,...,...,...,...,...
54019,2021-02-28,1900,9,14,61,ENE,[Clear],0.0,71,1034,9,7
54020,2021-02-28,2000,8,14,65,ENE,[Clear],0.0,74,1034,8,6
54021,2021-02-28,2100,8,14,69,ENE,[Clear],0.0,78,1034,8,5
54022,2021-02-28,2200,7,13,72,ENE,[Clear],0.0,79,1034,7,5


In [101]:
# remove data outside of january 5 2015 to december 31 2020
weather['date'] = pd.to_datetime(weather['date'])
start_date = '2015-1-4'
end_date = '2020-12-31'
mask = (weather['date'] > start_date) & (weather['date'] <= end_date)
weather = weather.loc[mask]

In [102]:
# no NaN
weather[weather.isnull().any(axis=1)]

,date,time,tempC,windspeedKmph,winddirDegree,winddir16Point,weatherDesc,precipMM,humidity,pressure,HeatIndexC,FeelsLikeC


In [ ]:
# other preprocessing
# - convert non-numerical variables to numerical
# - set datetime same between day ahead and weather